In [81]:
import numpy as np
from ase import Atoms
from stemplot.interactive import imshow
from mtflearn.io import load_image
from skimage.transform import rotate
%matplotlib qt

In [1]:
def get_a_thickness(formula):
    a_thickness_dict = {
        'TiS2': (3.35, 3.011),
        'VS2': (3.19, 2.962),
        'NbS2': (3.37, 3.397),
        'NbSe2': (3.5, 3.343),
        'MoS2': (3.19, 3.128),
        'MoSe2': (3.32, 3.31),
        'MoTe2': (3.57, 3.579),
        'TaS2': (3.34, 3.099),
        'TaSe2': (3.47, 3.331),
        'WS2': (3.18, 3.12),
        'WSe2': (3.32, 3.324),
        'WTe2': (3.56, 3.578),
        'ReSe2': (3.47, 3.1)
    }
    try:
        a, t = a_thickness_dict[formula]
    except:
        a, t = 3.19, 3.128
    return a, t


def mx2_unit(formula='MoS2', size=(1, 1, 1), vacuum=2):
    # get a and thickness according to formula
    a, thickness = get_a_thickness(formula)

    basis = [(0, 0, 0),
             (2 / 3, 1 / 3, 0.5 * thickness),
             (2 / 3, 1 / 3, -0.5 * thickness)]
    cell = [[a, 0, 0], [-a / 2, a * 3 ** 0.5 / 2, 0], [0, 0, 0]]

    atoms = Atoms(formula, cell=cell, pbc=(1, 1, 0))
    atoms.set_scaled_positions(basis)
    if vacuum is not None:
        atoms.center(vacuum, axis=2)
    atoms = atoms.repeat(size)
    return atoms


def make_it_orhto(atoms):
    cell_ = atoms.cell.copy()
    a, b, c = atoms.cell
    cell_[1] = [0., b[1], 0.]

    atoms.set_cell(cell_)
    atoms.wrap()
    atoms.center()
    return atoms

In [3]:
import abtem

def run_adf(atoms, energy=80, semiangle_cutoff=30, inner=70, outer=200,
            sampling=0.05, sampling_scan = None,focal_spread = False,nyquist_sampling_ratio = 0.5,
            laadf = True,laadf_inner = 50,laadf_outer = 120):
    # potential
    potential = abtem.Potential(atoms, sampling=sampling)
    energy = energy*1000
    if focal_spread:
        defocus_distribution = abtem.distributions.gaussian(
            center=0.0,
            standard_deviation=20,
            num_samples=11,
            sampling_limit=2,
            ensemble_mean=False,
        )
        probe = abtem.Probe(energy=energy, semiangle_cutoff=semiangle_cutoff,defocus=defocus_distribution)
    else:
        probe = abtem.Probe(energy=energy, semiangle_cutoff=semiangle_cutoff)

    # probe
    #     probe = abtem.Probe(energy=energy, semiangle_cutoff=semiangle_cutoff,defocus=defocus_distribution)
    probe.grid.match(potential)
    cutoff_angle = min(probe.cutoff_angles)
    assert cutoff_angle > outer, 'probe cutoff angle is {}, and is smaller than detector outer angle {}.'.format(cutoff_angle, outer)

    # detector
    detector_haadf = abtem.AnnularDetector(inner=inner, outer=outer)
    detector_laadf = abtem.AnnularDetector(inner=laadf_inner,outer=laadf_outer)
    #     scan: interaction bwtween probe and potential
    if sampling_scan is None:
        sampling_scan = probe.aperture.nyquist_sampling * nyquist_sampling_ratio

    if laadf:
        all_detectors = [detector_laadf,detector_haadf]
    else:
        all_detectors = [detector_haadf]
    print(sampling_scan)
    grid_scan = abtem.GridScan(
        start=[0, 0],
        end=[1, 1],
        sampling=[sampling_scan, sampling_scan],
        fractional=True,
        potential=potential,)
    measurements = probe.scan(potential, detectors=all_detectors, scan=grid_scan).compute()
    return measurements

In [25]:
def concatenate_images(images, n, m):
    num_images, height, width = images.shape
    if n * m > num_images:
        raise ValueError("The product of n and m is larger than the number of images.")

    large_image = np.zeros((height * m, width * n), dtype=images.dtype)

    for i in range(m):
        for j in range(n):
            index = i * n + j
            if index < num_images:
                row_start = i * height
                row_end = row_start + height
                col_start = j * width
                col_end = col_start + width

                large_image[row_start:row_end, col_start:col_end] = images[index]

    return large_image

In [88]:
def crop_image(image, size):
    height, width = image.shape

    start_y = (height - size) // 2
    start_x = (width - size) // 2

    cropped_image = image[start_y:start_y + size, start_x:start_x + size]

    return cropped_image

In [2]:
formula = 'WSe2'
energy = 80
structure = mx2_unit(formula,size = (1,2,1))
structure = make_it_orhto(structure)

In [5]:
sampling = 0.194966 #Angstrom/pixel

In [65]:
haadf = run_adf(atoms = structure, energy = energy,laadf = False, sampling_scan = sampling)

0.194966
[########################################] | 100% Completed | 2.73 sms


In [84]:
g_haadf = haadf.gaussian_filter(0.45)
p_haadf = g_haadf.poisson_noise(dose_per_area=1e4,samples = 5000)

In [85]:
p_haadf_img = p_haadf.array

In [86]:
l_img = concatenate_images(p_haadf_img, 60,75)
imshow(l_img)

In [94]:
r_img = rotate(l_img,angle = -12)
# imshow(r_img)

In [95]:
c_img = crop_image(r_img,1024)
imshow(c_img)

In [97]:
np.save('croped simu image.npy',c_img)

In [79]:
exp_img = load_image('ADF2_27.dm4')

In [96]:
imshow(exp_img)

In [103]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots(1,2,figsize = (50,20))
ax[0].imshow(c_img)
ax[1].imshow(exp_img)
ax[0].axis('off')
ax[1].axis('off')
plt.savefig('simu and exp.jpg',dpi = 500,transparent=True)